In [1]:
# pip install scikit-surprise

In [2]:
import pandas as pd
from surprise import SVD
from surprise import Dataset
from surprise.model_selection import cross_validate
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report


In [3]:
df=pd.read_csv(r'C:\Users\BK\OneDrive\Desktop\Assignment\2019-Oct.csv')
df_users=pd.read_csv(r'C:\Users\BK\OneDrive\Desktop\Assignment\Ben_final.csv')

In [4]:
df_cluster=pd.read_csv(r'C:\Users\BK\OneDrive\Desktop\Assignment\Final Group Assignments.csv')


In [5]:
def clusters(df_cluster,df_users):

    cluster0 = pd.merge(df_cluster[['Group 0']], df_users,  how='right', left_on=['Group 0'], right_on = ['user_id'])
    cluster0=cluster0.dropna()

    cluster1 = pd.merge(df_cluster[['Group 1']], df_users,  how='right', left_on=['Group 1'], right_on = ['user_id'])
    cluster1=cluster1.dropna()

    cluster2 = pd.merge(df_cluster[['Group 2']], df_users,  how='right', left_on=['Group 2'], right_on = ['user_id'])
    cluster2=cluster2.dropna()
    return cluster0,cluster1,cluster2

In [6]:
cluster0,cluster1,cluster2=clusters(df_cluster,df_users)
cluster0[['avg_num_purchase_per_transaction','avg_purchase_price_per_user']].mean(),cluster1[['avg_num_purchase_per_transaction','avg_purchase_price_per_user']].mean(),cluster2[['avg_num_purchase_per_transaction','avg_purchase_price_per_user']].mean()


(avg_num_purchase_per_transaction      1.105786
 avg_purchase_price_per_user         180.422646
 dtype: float64,
 avg_num_purchase_per_transaction      1.157228
 avg_purchase_price_per_user         396.172541
 dtype: float64,
 avg_num_purchase_per_transaction     1.107734
 avg_purchase_price_per_user         90.529917
 dtype: float64)

In [7]:
df_users1=df_users[['user_id']]

In [8]:
df2=df[['user_id','category_id','event_type']]

In [9]:
df_final=df2.merge(df_users1,how='left',on='user_id')

In [10]:
def create_dataset(df_cluster,df):
    df2=df[['user_id','category_id','event_type']]
#     df_final=df2.merge(df_cluster[['Group 0']],how='left',on='user_id')
    df_final_0= pd.merge(df_cluster[['Group 0']], df2,  how='right', left_on=['Group 0'], right_on = ['user_id'])
    df_final_0=df_final_0.dropna()

    df_final_1= pd.merge(df_cluster[['Group 1']], df2,  how='right', left_on=['Group 1'], right_on = ['user_id'])
    df_final_1=df_final_1.dropna()

    df_final_2= pd.merge(df_cluster[['Group 2']], df2,  how='right', left_on=['Group 2'], right_on = ['user_id'])
    df_final_2=df_final_2.dropna()

    df_list=[df_final_0,df_final_1,df_final_2]
    for dfs in df_list:
        dfs.loc[dfs['event_type']=='view','event_type'] = 1
        dfs.loc[dfs['event_type']=='cart','event_type'] = 2
        dfs.loc[dfs['event_type']=='purchase','event_type'] =2

#         dfs.loc[dfs['event_type']=='purchase','event_type'] =3
        dfs.rename(columns={'event_type':'ratings'},inplace=True)
    return df_final_0,df_final_1,df_final_2
    
    

In [11]:
# df_cluster
# df2
# a1=pd.merge(df_cluster[['Group 0']], df2,  how='right', left_on=['Group 0'], right_on = ['user_id'])
df_final_0,df_final_1,df_final_2=create_dataset(df_cluster,df)




## UNBALANCED<----------------------------

df_final_0 = df_final_0.groupby('ratings').apply(lambda x: x.sample(n=140000)).reset_index(drop = True)
df_final_1 = df_final_1.groupby('ratings').apply(lambda x: x.sample(n=930000)).reset_index(drop = True)
df_final_2 = df_final_2.groupby('ratings').apply(lambda x: x.sample(n=1500)).reset_index(drop = True)






# df_final_0 = df_final_0.drop(df_final_0[df_final_0['category_id'] ==2053013555631882655].sample(frac=.3).index)

In [12]:
# df_final_0,df_final_1,df_final_2=create_dataset(df_cluster,df)

# df_final_1.ratings.value_counts()
# df_final_0.info()

In [13]:
from surprise import Reader

# df_final_0,df_final_1,df_final_2=create_dataset(df_cluster,df)
final_list=[df_final_0,df_final_1,df_final_2]

test_list=[]
train_list=[]
for x in final_list:
    train,test = train_test_split(x,test_size=0.3, random_state=42)
    test_list.append(test)
    train_list.append(train)

    
  


data_list=[]
def dateset_prep():
    reader = Reader(rating_scale=(1,2))

    for x in range(len(train_list)):
#     train,test = train_test_split(df_test,test_size=0.3, random_state=42)
#     test_list.append(test)
#     train.append(train)
        data = Dataset.load_from_df(train_list[x][['user_id', 'category_id', 'ratings']], reader)
        data_list.append(data)
    
    return data_list

data=dateset_prep()


In [14]:
data0=data[0]
data1=data[1]
data2=data[2]





In [15]:
models= []
for x in data:
    svd = SVD(verbose=True, n_epochs=5)
    cross_validate(svd, x, measures=['RMSE', 'MAE'], cv=3, verbose=True)
    trainset = x.build_full_trainset()
    svd.fit(trainset)
    models.append(svd)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.4766  0.4772  0.4775  0.4771  0.0004  
MAE (testset)     0.4520  0.4536  0.4540  0.4532  0.0009  
Fit time          2.61    2.53    2.78    2.64    0.10    
Test time         0.69    0.77    0.77    0.75    0.04    
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Proce

In [16]:
recomendation_all=[]

for test,model in zip(test_list,models):

    recomendation_list=[]
    id_sample=set(test.user_id)
    cat_ids=set(test.category_id)

    # user_list=[]
    for user in id_sample:
        for cat in cat_ids:
            predict=model.predict(uid=user, iid=cat)
    #         if predict.est>1.3:
            recomendation_list.append([user,cat,predict.est])
    recomendation_all.append(recomendation_list)


In [17]:
# id_sample=set(test_list[0].category_id)
# cat_ids=set(test_list[0].category_id)

In [18]:
eval_df_list=[]

for x,recomendation_list in zip(range(3),recomendation_all):

    pred_df = pd.DataFrame (recomendation_list, columns = ['user_id','category_id','pred_rating'])
    pred_df

    # test_list[0].drop(['Group 0'], axis=1,inplace=True)
    # test_list[1].drop(['Group 1'], axis=1,inplace=True)
    # test_list[2].drop(['Group 2'], axis=1,inplace=True)




    # sample=test_list.loc[test_list.user_id.isin(id_sample.values)]


    eval_df = pd.merge(pred_df,test_list[x],how='right', left_on=['user_id','category_id'], right_on = ['user_id','category_id'])
    eval_df_list.append(eval_df)


In [19]:
test_list[0]
eval_df

,user_id,category_id,pred_rating,Group 2,ratings
0,551383040,2053013562292437791,1.798578,551383040.0,2
1,514520579,2053013562292437791,1.689843,514520579.0,2
2,514520579,2053013555631882655,1.673723,514520579.0,1
3,514520579,2053013555631882655,1.673723,514520579.0,1
4,514520579,2053013555631882655,1.673723,514520579.0,1
...,...,...,...,...,...
895,535224305,2053013555631882655,1.629926,535224305.0,2
896,532240886,2053013562292437791,1.753172,532240886.0,2
897,537026551,2053013562292437791,1.806615,537026551.0,2
898,516707835,2053013562292437791,1.737086,516707835.0,2


In [42]:
final_evaluation_list=[]
for eval_df in eval_df_list:

    evalution_df=eval_df.copy()
#     evalution_df['pred_rating']=np.round(evalution_df.pred_rating).astype('Int64')
    evalution_df[evalution_df['pred_rating'] > 1.5] = 2
    evalution_df[evalution_df['pred_rating'] <= 1.5] = 1
    
    evalution_df['ratings']=evalution_df.ratings.astype('Int64')
    # evalution_df=evalution_df.loc[evalution_df.ratings!=1]
    # evalution_df.info()                                                              

    final_evaluation_list.append(evalution_df)


In [43]:
y=list(final_evaluation_list[0].ratings.values)
ypred=list(final_evaluation_list[0].pred_rating.values)
print(classification_report(y, ypred))


              precision    recall  f1-score   support

           1       1.00      1.00      1.00     35798
           2       1.00      1.00      1.00     48202

    accuracy                           1.00     84000
   macro avg       1.00      1.00      1.00     84000
weighted avg       1.00      1.00      1.00     84000



In [37]:
y1=list(final_evaluation_list[1].ratings.values)
ypred1=list(final_evaluation_list[1].pred_rating.values)
print(classification_report(y1, ypred1))


              precision    recall  f1-score   support

           1       1.00      1.00      1.00    121664
           2       1.00      1.00      1.00    436336

    accuracy                           1.00    558000
   macro avg       1.00      1.00      1.00    558000
weighted avg       1.00      1.00      1.00    558000



In [23]:
y2=list(final_evaluation_list[2].ratings.values)
ypred2=list(final_evaluation_list[2].pred_rating.values)
print(classification_report(y2, ypred2))


              precision    recall  f1-score   support

           1       0.75      0.50      0.60       461
           2       0.61      0.82      0.70       439

    accuracy                           0.66       900
   macro avg       0.68      0.66      0.65       900
weighted avg       0.68      0.66      0.65       900



### EXTRA BELOW

In [24]:
test_0=final_evaluation_list[0].loc[final_evaluation_list[0]['pred_rating']==2]


test_sample0 = pd.merge(test_0,df,how='inner', left_on=['user_id','category_id'], right_on = ['user_id','category_id'])
test_sample0



#----------------------------------------------------------


test_1=final_evaluation_list[1].loc[final_evaluation_list[1]['pred_rating']==2]


test_sample1 = pd.merge(test_1,df,how='inner', left_on=['user_id','category_id'], right_on = ['user_id','category_id'])
test_sample1

#----------------------------------------------------------



test_2=final_evaluation_list[2].loc[final_evaluation_list[2]['pred_rating']==2]


test_sample2 = pd.merge(test_2,df,how='inner', left_on=['user_id','category_id'], right_on = ['user_id','category_id'])
test_sample2



,user_id,category_id,pred_rating,Group 2,ratings,event_time,event_type,product_id,category_code,brand,price,user_session
0,551383040,2053013562292437791,2,551383040.0,2,2019-10-23 09:41:23 UTC,view,25800005,medicine.tools.tonometer,omron,40.90,e79a70ec-448f-4a54-b2df-0917736bc145
1,551383040,2053013562292437791,2,551383040.0,2,2019-10-23 09:44:51 UTC,view,25800005,medicine.tools.tonometer,omron,40.90,e79a70ec-448f-4a54-b2df-0917736bc145
2,551383040,2053013562292437791,2,551383040.0,2,2019-10-23 09:46:59 UTC,view,25800007,medicine.tools.tonometer,omron,47.59,e79a70ec-448f-4a54-b2df-0917736bc145
3,551383040,2053013562292437791,2,551383040.0,2,2019-10-23 09:52:39 UTC,view,25800007,medicine.tools.tonometer,omron,47.59,88f7dcb7-2501-4643-b016-97ad664bae5a
4,551383040,2053013562292437791,2,551383040.0,2,2019-10-23 10:26:55 UTC,view,25800007,medicine.tools.tonometer,omron,47.59,5e6c0513-60eb-4591-8cc5-ce4c3d873471
...,...,...,...,...,...,...,...,...,...,...,...,...
40400,516707835,2053013562292437791,2,516707835.0,2,2019-10-22 06:35:53 UTC,view,25800005,medicine.tools.tonometer,omron,40.90,8f8f5a49-5f89-4b4a-a60c-2e3953218f5f
40401,516707835,2053013562292437791,2,516707835.0,2,2019-10-22 06:37:04 UTC,cart,25800005,medicine.tools.tonometer,omron,40.90,8f8f5a49-5f89-4b4a-a60c-2e3953218f5f
40402,516707835,2053013562292437791,2,516707835.0,2,2019-10-22 06:39:09 UTC,purchase,25800005,medicine.tools.tonometer,omron,40.90,8f8f5a49-5f89-4b4a-a60c-2e3953218f5f
40403,516707835,2053013553559896355,2,516707835.0,2,2019-10-11 15:56:26 UTC,view,12708082,NaN,matador,59.20,3ab2539b-a542-4bc9-8820-11689a474975


In [25]:
test_sample0.price.mean(),test_sample1.price.mean(),test_sample2.price.mean()



# .loc[test_sample2['pred_rating']]

(298.16122785569644, 475.8267597466473, 391.5602502165639)

In [26]:
final_evaluation_list[2].loc[final_evaluation_list[2]['pred_rating']==2]


,user_id,category_id,pred_rating,Group 2,ratings
0,551383040,2053013562292437791,2,551383040.0,2
1,514520579,2053013562292437791,2,514520579.0,2
2,514520579,2053013555631882655,2,514520579.0,1
3,514520579,2053013555631882655,2,514520579.0,1
4,514520579,2053013555631882655,2,514520579.0,1
...,...,...,...,...,...
895,535224305,2053013555631882655,2,535224305.0,2
896,532240886,2053013562292437791,2,532240886.0,2
897,537026551,2053013562292437791,2,537026551.0,2
898,516707835,2053013562292437791,2,516707835.0,2


## CONFUSION MATRIX 

## PREDICTED MIGHT PREDICT CATEGORY ID'S THAT SOME USERS MAY NOT ORIGINALLY HAVE TAKE THAT INTO ACCOUNT FOR EVALUATION

## Unclustered

In [27]:
df_users1=df_users[['user_id']]
df2=df[['user_id','category_id','event_type']]
df_final=df2.merge(df_users1,how='left',on='user_id')
df_final.loc[df_final['event_type']=='view','event_type'] = 1
df_final.loc[df_final['event_type']=='cart','event_type'] = 2
df_final.loc[df_final['event_type']=='purchase','event_type'] =2
df_final.rename(columns={'event_type':'ratings'},inplace=True)
df_final2 = df_final.groupby('ratings').apply(lambda x: x.sample(n=250000)).reset_index(drop = True)




In [28]:
final_train,final_test = train_test_split(df_final2,test_size=0.3, random_state=42)



reader = Reader(rating_scale=(1,2))
final_data = Dataset.load_from_df(final_train[['user_id', 'category_id', 'ratings']], reader)
svd2 = SVD(verbose=True, n_epochs=5)
cross_validate(svd2, final_data, measures=['RMSE', 'MAE'], cv=3, verbose=True)
trainset2 = final_data.build_full_trainset()
svd2.fit(trainset2)

Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4
Evaluating RMSE, MAE of algorithm SVD on 3 split(s).

                  Fold 1  Fold 2  Fold 3  Mean    Std     
RMSE (testset)    0.4638  0.4643  0.4636  0.4639  0.0003  
MAE (testset)     0.4376  0.4374  0.4383  0.4378  0.0004  
Fit time          5.07    6.68    4.80    5.52    0.83    
Test time         0.78    0.93    0.82    0.84    0.06    
Processing epoch 0
Processing epoch 1
Processing epoch 2
Processing epoch 3
Processing epoch 4


In [29]:
recomendation_list_final=[]
# user_list=[]
id_sample = set(final_test.user_id)
cat_ids=set(final_test.category_id)

for user in id_sample:
    for cat in cat_ids:
        predict=svd2.predict(uid=user, iid=cat)
#         if predict.est>1.3:
        recomendation_list_final.append([user,cat,predict.est])


In [30]:
recomendation_list_final


[[515112961, 2075962323838697472, 1.6279273660892761],
 [515112961, 2053013566344134657, 1.4219724891258485],
 [515112961, 2116907504691576833, 1.268602518579298],
 [515112961, 2053013566377689091, 1.4757473388010625],
 [515112961, 2053013566411243525, 1.259524777528903],
 [515112961, 2053013566453186567, 1.2762778321385557],
 [515112961, 2134905045102100487, 1.4128357799070583],
 [515112961, 2053013566478352393, 1.29826314838257],
 [515112961, 2134905045177597961, 1.2883361946290774],
 [515112961, 2134905045253095435, 1.4854494950446984],
 [515112961, 2134905045328592909, 1.5758346815570297],
 [515112961, 2053013566537072653, 1.3650305394088678],
 [515112961, 2053013566562238479, 1.1239610440296572],
 [515112961, 2053013566587404305, 1.1782500802276843],
 [515112961, 2152167773222993940, 1.1680524485281063],
 [515112961, 2134905045613805589, 1],
 [515112961, 2144916515806248980, 1.3273482304138318],
 [515112961, 2074462942123786261, 1.5262847606443348],
 [515112961, 208949011611962574

In [31]:
# eval_df_list=[]

# for x,recomendation_list in zip(range(3),recomendation_all):

pred_df_final = pd.DataFrame (recomendation_list_final, columns = ['user_id','category_id','pred_rating'])
pred_df_final

    # test_list[0].drop(['Group 0'], axis=1,inplace=True)
    # test_list[1].drop(['Group 1'], axis=1,inplace=True)
    # test_list[2].drop(['Group 2'], axis=1,inplace=True)




    # sample=test_list.loc[test_list.user_id.isin(id_sample.values)]


eval_df2 = pd.merge(pred_df_final,final_test,how='right', left_on=['user_id','category_id'], right_on = ['user_id','category_id'])
# eval_df_list.append(eval_df)


In [32]:
evalution_df2=eval_df2.copy()
evalution_df2['pred_rating']=np.round(evalution_df2.pred_rating).astype('Int64')
evalution_df2['ratings']=evalution_df2.ratings.astype('Int64')
# evalution_df=evalution_df.loc[evalution_df.ratings!=1]
# evalution_df.info()                                                              

# final_evaluation_list.append(evalution_df)


In [33]:
y=list(evalution_df2.ratings.values)
ypred=list(evalution_df2.pred_rating.values)
print(classification_report(y, ypred))

              precision    recall  f1-score   support

           1       0.67      0.66      0.67     75264
           2       0.66      0.68      0.67     74736

    accuracy                           0.67    150000
   macro avg       0.67      0.67      0.67    150000
weighted avg       0.67      0.67      0.67    150000



In [34]:
test_0=final_evaluation_list[0].loc[final_evaluation_list[0]['pred_rating']==2]


test_sample0 = pd.merge(test_0,df,how='inner', left_on=['user_id','category_id'], right_on = ['user_id','category_id'])
test_sample0

,user_id,category_id,pred_rating,Group 0,ratings,event_time,event_type,product_id,category_code,brand,price,user_session
0,535166990,2053013563911439225,2,535166990.0,1,2019-10-01 13:22:37 UTC,view,2702538,appliances.kitchen.refrigerators,dauscher,200.52,2519549d-cdb4-4975-bfb7-a8c4f78b18bb
1,535166990,2053013563911439225,2,535166990.0,1,2019-10-01 13:24:01 UTC,view,2702538,appliances.kitchen.refrigerators,dauscher,200.52,2519549d-cdb4-4975-bfb7-a8c4f78b18bb
2,535166990,2053013563911439225,2,535166990.0,1,2019-10-01 13:25:11 UTC,view,2702538,appliances.kitchen.refrigerators,dauscher,200.52,2519549d-cdb4-4975-bfb7-a8c4f78b18bb
3,535166990,2053013563911439225,2,535166990.0,1,2019-10-01 13:25:48 UTC,view,2702538,appliances.kitchen.refrigerators,dauscher,200.52,2519549d-cdb4-4975-bfb7-a8c4f78b18bb
4,535166990,2053013563911439225,2,535166990.0,1,2019-10-01 13:26:23 UTC,view,2702538,appliances.kitchen.refrigerators,dauscher,200.52,2519549d-cdb4-4975-bfb7-a8c4f78b18bb
...,...,...,...,...,...,...,...,...,...,...,...,...
2373719,538050553,2053013555631882655,2,538050553.0,1,2019-10-31 12:00:44 UTC,view,1003864,electronics.smartphone,oppo,231.67,157b0832-9207-4032-9186-c88de8200b6a
2373720,538050553,2053013555631882655,2,538050553.0,1,2019-10-31 12:01:15 UTC,view,1004121,electronics.smartphone,meizu,257.15,157b0832-9207-4032-9186-c88de8200b6a
2373721,538050553,2053013555631882655,2,538050553.0,1,2019-10-31 12:01:48 UTC,view,1005002,electronics.smartphone,huawei,244.51,157b0832-9207-4032-9186-c88de8200b6a
2373722,538050553,2053013555631882655,2,538050553.0,1,2019-10-31 12:02:03 UTC,view,1005002,electronics.smartphone,huawei,244.51,157b0832-9207-4032-9186-c88de8200b6a
